In [1]:
import numpy as np
import pcabm.dcbm as dc
import pcabm.pcabm as pca
import pcabm.sc as sc
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score

In [2]:
## Parameter Settings
p, n, k = 5,200,2
rho = 2*np.log(n)**1.5/n                  # sparsity level
gamma = 1*np.array((0.4,0.8,1.2,1.6,2))   # covariate coefficients

## Generate Covariates
X = np.zeros((n,p))
X[:,0] = np.random.binomial(1,0.1,n)
X[:,1] = np.random.poisson(0.1,n)
X[:,2] = np.random.uniform(0,1,n)
X[:,3] = np.random.exponential(0.3,n)
X[:,4] = np.random.normal(0,0.3,n)

## Generate Pairwise Covariates
Z = np.zeros((n,n,p))
for i in np.arange(p):
    temp1 = np.repeat(X[:,i],repeats=n,axis=0)
    temp2 = np.reshape(temp1, (n, n), order='F')
    temp3 = np.matrix.transpose(temp2)
    Z[:,:,i] = abs(temp2-temp3)

# Generate Edge
B = np.ones((n,n));B[0:(n//2),0:(n//2)]=2;B[(n//2):n,(n//2):n]=2;B=B*rho
lam = np.multiply(B,np.exp(np.dot(Z,gamma)))
A = np.random.poisson(np.tril(lam,-1)); Ab = A+A.T
gt=np.array([0]*(n//2)+[1]*(n//2))      # ground truth

# PCABM.SCWA

In [7]:
modelSCWA,gamma = sc.SCWA(Ab,Z,2)
print('ARI is', adjusted_rand_score(modelSCWA.labels_,gt))
print('NMI is', normalized_mutual_info_score(modelSCWA.labels_,gt))

ARI is 1.0
NMI is 1.0


# PCABM.MLE0

In [5]:
# This may give a bad result. Usually, we need to try multiple initializations.
modelPCA = pca.PCABM(Ab,Z,2,gamma)
estPCA = modelPCA.fit(community=np.random.randint(2, size=n),gt=gt)

ARI is 1.0
NMI is 1.0


#  PCABM.MLE

In [8]:
modelPCA = pca.PCABM(Ab,Z,2,gamma)
estPCA = modelPCA.fit(community=modelSCWA.labels_,gt=gt)

ARI is 1.0
NMI is 1.0


# SBM.SC

In [11]:
modelSC = sc.SC(Ab,2)
print('ARI is', adjusted_rand_score(modelSC.labels_,gt))
print('NMI is', normalized_mutual_info_score(modelSC.labels_,gt))

ARI is 1.0
NMI is 1.0


# SCORE

In [12]:
modelSCORE = sc.SCORE(Ab,2)
print('ARI is', adjusted_rand_score(modelSCORE.labels_,gt))
print('NMI is', normalized_mutual_info_score(modelSCORE.labels_,gt))

ARI is -0.0023146359825917348
NMI is 0.0019185362230944743
